In [28]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import json

import pymongo
from pymongo import MongoClient

# Census & gmaps API Keys
# from config import (api_key, gkey)
# c = Census(api_key, year=2013)

# # Configure gmaps
# gmaps.configure(api_key=gkey)

ModuleNotFoundError: No module named 'census'

In [29]:
# Making a Connection with MongoClient
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.Crimes_db

# Declare the collection
HighUnemp = db.HighUnemp
HighViolent = db.HighViolent
HighProperty = db.HighProperty
HighCrime = db.HighCrime
UnEmp10 = db.UnEmp10
ViolentTotal10 = db.ViolentTotal10
PropertyTotal10= db.PropertyTotal10
CrimeRate10 = db.CrimeRate10

NameError: name 'pymongo' is not defined

In [32]:
state
statecsv= pd.read_csv("./data/statelatlong.csv")
statecsv.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\statelatlong.csv'

In [25]:
crimestatecsv= pd.read_csv("./data/crimebystatecombinedwithunemployment (1).csv")
crimestatecsv.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/crimebystatecombinedwithunemployment (1).csv'

In [26]:
# Merge the datasets using the state columns, adding the latitude and longitude columns
crime_state_data = pd.merge(crimestatecsv,statecsv, how="left", left_on="state", right_on="State")

# Save the updated dataframe as a csv
crime_state_data.to_csv("./Unemp_crime_state_data.csv", encoding="utf-8", index=False)
crime_state_data.head()

NameError: name 'crimestatecsv' is not defined

In [6]:
#filtering the data for 10 years from 2004 to 2014

newdf = crime_state_data[crime_state_data.year.isin([2004,2005,2006,2007,2008,2009, 2010,2011,2012,2013,2014])]
newdf.head()

NameError: name 'crime_state_data' is not defined

In [7]:
# new = old[['A','C','D']]
newdf2 = newdf[['state','unemployment','year','Population','Murder','rape','Robbery','Aggravated assault'
               ,'violent total','Burglary','Larceny theft','vehicle theft','property total',
               'Latitude','Longitude']]
newdf2

NameError: name 'newdf' is not defined

In [8]:
newdf2.columns=['State','Unemployment','Year','Population','Murder','Rape','Robbery','Aggravated Assault'
               ,'Violent Total','Burglary','Larceny Theft','Vehicle Theft','Property Total',
               'Latitude','Longitude']
newdf2

NameError: name 'newdf2' is not defined

In [9]:
newdf2=newdf2.dropna(how='any')

NameError: name 'newdf2' is not defined

In [ ]:
# Adding a new column for the Violent and Property Crime
newdf2['Total_CrimeRate'] = newdf2['Violent Total'] + newdf2['Property Total']
newdf2

In [ ]:
# newdf_topUnemp_Year = newdf2['State','Unemployment','Year']

newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)
newdf_topUnemp_Year 

# print(newdf_topUnemp_Year.groupby('Year').
newdf_topUnemp_Year.groupby('Year')['Unemployment'].max().reset_index()

In [ ]:
# Highest Unemployment rated states from 2004 - 2014  HighUnemp
newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)


newdf3=newdf_topUnemp_Year2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Unemployment", ascending=False)
.head(1)).droplevel(0).sort_values("Unemployment", ascending=False)

# newdf2[['State','Unemployment','Year']].groupby("Year", as_index=False).apply(lambda df:df.sort_values("Unemployment", ascending=False)
# .head(1)).droplevel(0).sort_values("Unemployment", ascending=False)

In [ ]:
records = json.loads(newdf3.T.to_json()).values()
db.HighUnemp.insert_many(records)

# db.HighUnemp.insert_many(newdf3)

In [ ]:
# Highest Violent Total for states from 2004 - 2014  HighViolent
newdf_topUnemp_Year = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Violent Total", ascending=False)
.head(1)).droplevel(0).sort_values("Violent Total", ascending=False)

In [ ]:

# HighProperty = db.HighProperty
# HighCrime = db.HighCrime
# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighViolent.insert_many(records)

In [ ]:
# Highest Property Total for states from 2004 - 2014 - HighProperty
newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Property Total", ascending=False)
.head(1)).droplevel(0).sort_values("Property Total", ascending=False)

In [ ]:
# HighCrime = db.HighCrime
# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighProperty.insert_many(records)

In [ ]:
# Highest CrimeRate (Violent+Property) Total for states from 2004 - 2014  - HighCrime
newdf3=newdf2.groupby("Year", as_index=False).apply(lambda df:df.sort_values("Total_CrimeRate", ascending=False)
.head(1)).droplevel(0).sort_values("Total_CrimeRate", ascending=False)

In [ ]:

# UnEmp10 = db.UnEmp10
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.HighCrime.insert_many(records)

In [ ]:
# 10 highest Unemployment rated states sorted for each year - UnEmp10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf_topUnemp_Year2.sort_values(by=['Year','Unemployment'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [ ]:

# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.UnEmp10.insert_many(records)

In [ ]:
# 10 highest Violent Totalstates sorted for each year - ViolentTotal10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Violent Total'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [ ]:
# ViolentTotal10 = db.ViolentTotal10
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.ViolentTotal10.insert_many(records)

In [ ]:
# 10 highest Property Total for states sorted for each year -PropertyTotal10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Property Total'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [ ]:
# PropertyTotal10= db.PropertyTotal10
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.PropertyTotal10.insert_many(records)

In [ ]:
# 10 highest Total Crime for states sorted for each year -CrimeRate10

newdf_topUnemp_Year2 = newdf2.filter(['State','Unemployment','Year'], axis=1)

newdf3=newdf2.sort_values(by=['Year','Total_CrimeRate'], ascending=False).groupby('Year').head(10)
# newdf_topUnemp_Year2.groupby('Year').head(10)

In [ ]:
# CrimeRate10 = db.CrimeRate10

records = json.loads(newdf3.T.to_json()).values()
db.CrimeRate10.insert_many(records)

In [ ]:
# converting the dataframe to json object and saved into Export_DataFrame.json file.
# js_all = 
newdf2.to_json(r'Export_DataFrame.json',orient = 'records')
# print(js_all)

In [ ]:
# Function to convert the dataframe to geojson data.

def df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [ ]:
cols = ['State','Unemployment','Year','Population','Murder','Rape','Robbery','Aggravated Assault'
               ,'Violent Total','Burglary','Larceny Theft','Vehicle Theft','Property Total','Total_CrimeRate']
# geojson = df_to_geojson(df_geo, cols)

# calling the function to convert the dataframe to geojson data.

geojson = df_to_geojson(newdf2, cols)

In [ ]:
# save the geojson result to a file  'dataset.js'
output_filename = 'dataset.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson)))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson['features'])))